# CSE 158 HW 2
#### Hongtao Jiang A13760857
#### 10/22/19

Before answering the questions, I imported all the models that will be used.

In [1]:
import pandas as pd
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm
from sklearn import linear_model
from urllib.request import urlopen
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.io import arff
from sklearn import metrics
from sklearn.decomposition import PCA

Firstly, I imported the data。

In [2]:
f = open("5year.arff", 'r')
while not '@data' in f.readline():
    pass
dataset = []
for l in f:
    if '?' in l: # Missing entry
        continue
    l = l.split(',')
    values = [1] + [float(x) for x in l]
    values[-1] = values[-1] > 0 # Convert to bool
    dataset.append(values)

In [3]:
X = [values[:-1] for values in dataset]
y = [values[-1] for values in dataset]

In [4]:
import warnings
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=ConvergenceWarning)

## Question 1

Question: Train a logistic regressor (e.g. sklearn.linear model.LogisticRegression) with regularization coefficient C = 1.0. Report the accuracy and Balanced Error Rate (BER) of your classifier (1 mark).

#### Answer for Question 1:

In [5]:
model_1 = linear_model.LogisticRegression(C=1.0)
model_1.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
predictions = model_1.predict(X)

In [7]:
print('Accuracy Score:', metrics.accuracy_score(y, predictions))
print('Balanced Error Rate:', 1 - metrics.balanced_accuracy_score(y, predictions))

Accuracy Score: 0.9666776641372484
Balanced Error Rate: 0.48090427704028005


## Question 2

Question: Retrain the above model using the class weight=’balanced’ option. Report the accuracy and BER of your new classifier (1 mark).

#### Answer for Question 2:

In [8]:
model_2 = linear_model.LogisticRegression(C=1.0, class_weight='balanced')
model_2.fit(X, y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
predictions_2 = model_2.predict(X)

In [10]:
print('Accuracy Score:', 
      metrics.accuracy_score(y, predictions_2))
print('Balanced Error Rate:', 
      1 - metrics.balanced_accuracy_score(y, predictions_2))

Accuracy Score: 0.7806004618937644
Balanced Error Rate: 0.2034137997978297


## Question 3

Question: Shuffle the data, and split it into training, validation, and test splits, with a 50/25/25% ratio. Using the class weight=’balanced’ option, and training on the training set, report the training/validation/test accuracy and BER (1 mark).

#### Answer for Question 3:

In [107]:
import random
random.shuffle(dataset)

In [108]:
X = [values[:-1] for values in dataset]
y = [values[-1] for values in dataset]

In [109]:
X_train=X[:len(X)//2]
X_test=X[len(X)*3//4:]
X_vali=X[len(X)//2:len(X)*3//4] 
y_train=y[:len(X)//2]
y_test=y[len(X)*3//4:]
y_vali=y[len(X)//2:len(X)*3//4]

In [110]:
len(X), len(X_train), len(X_test), len(X_vali)

(3031, 1515, 758, 758)

In [111]:
model_3 = linear_model.LogisticRegression(C=1.0, class_weight='balanced')
model_3.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [112]:
train_predictions_3 = model_3.predict(X_train)
vali_predictions_3 = model_3.predict(X_vali)
test_predictions_3 = model_3.predict(X_test)

In [113]:
print('TRAIN Accuracy Score:', 
      metrics.accuracy_score(y_train, train_predictions_3))
print('TRAIN Balanced Error Rate:', 
      1 - metrics.balanced_accuracy_score(y_train, train_predictions_3))

TRAIN Accuracy Score: 0.7973597359735973
TRAIN Balanced Error Rate: 0.16273037542662117


In [114]:
print('VALIDATION Accuracy Score:', 
      metrics.accuracy_score(y_vali, vali_predictions_3))
print('VALIDATION Balanced Error Rate:', 
      1 - metrics.balanced_accuracy_score(y_vali, vali_predictions_3))

VALIDATION Accuracy Score: 0.7823218997361477
VALIDATION Balanced Error Rate: 0.31908023483365944


In [115]:
print('TEST Accuracy Score:', 
      metrics.accuracy_score(y_test, test_predictions_3))
print('TEST Balanced Error Rate:', 
      1 - metrics.balanced_accuracy_score(y_test, test_predictions_3))

TEST Accuracy Score: 0.7770448548812665
TEST Balanced Error Rate: 0.31664396003633066


## Question 4

Question: Implement a complete regularization pipeline. Consider values of C in the range {10−4, 10−3, . . . , 103, 104}. Report (or plot) the train, validation, and test BER for each value of C. Based on these values, which classifier would you select (in terms of generalization performance) and why (1 mark)?

#### Answer for Question 4:

In [116]:
train_predictions_lst = []
vali_predictions_lst = []
test_predictions_lst = []
bestBER = 1
bestModel = None
train_ber = []
test_ber = []
vali_ber = []

for num in [10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]:
    model = linear_model.LogisticRegression(C=num, class_weight='balanced')
    model.fit(X_train, y_train)
    
    train_predictions_lst.append(model.predict(X_train))
    vali_predictions_lst.append(model.predict(X_vali))
    test_predictions_lst.append(model.predict(X_test))
    
    train_ber.append(1 - metrics.balanced_accuracy_score(y_train, model.predict(X_train)))
    test_ber.append(1 - metrics.balanced_accuracy_score(y_test, model.predict(X_test)))
    vali_ber.append(1 - metrics.balanced_accuracy_score(y_vali, model.predict(X_vali)))
    
    print('TRAIN Accuracy Score for C = ' + str(num) +':', 
          metrics.accuracy_score(y_train, model.predict(X_train)))
    print('TRAIN Balanced Error Rate for C = ' + str(num) +':', 
          1 - metrics.balanced_accuracy_score(y_train, model.predict(X_train)))
    
    print('TEST Accuracy Score for C = ' + str(num) +':', 
          metrics.accuracy_score(y_test, model.predict(X_test)))
    print('TEST Balanced Error Rate for C = ' + str(num) +':', 
          1 - metrics.balanced_accuracy_score(y_test, model.predict(X_test)))
    
    print('VALIDATION Accuracy Score for C = ' + str(num) +':', 
          metrics.accuracy_score(y_vali, model.predict(X_vali)))
    print('VALIDATION Balanced Error Rate for C = ' + str(num) +':', 
          1 - metrics.balanced_accuracy_score(y_vali, model.predict(X_vali)))
    
    print('')
    
    if not bestModel or (1-metrics.balanced_accuracy_score(y_vali, model.predict(X_vali))) < bestBER:
        bestBER = 1 - metrics.balanced_accuracy_score(y_vali, model.predict(X_vali))
        bestModel = model
        bestC = num

TRAIN Accuracy Score for C = 0.0001: 0.7755775577557755
TRAIN Balanced Error Rate for C = 0.0001: 0.2512627986348124
TEST Accuracy Score for C = 0.0001: 0.7519788918205804
TEST Balanced Error Rate for C = 0.0001: 0.30943460490463215
VALIDATION Accuracy Score for C = 0.0001: 0.7612137203166227
VALIDATION Balanced Error Rate for C = 0.0001: 0.3987279843444227

TRAIN Accuracy Score for C = 0.001: 0.8092409240924092
TRAIN Balanced Error Rate for C = 0.001: 0.1759044368600683
TEST Accuracy Score for C = 0.001: 0.7928759894459103
TEST Balanced Error Rate for C = 0.001: 0.3084695731153497
VALIDATION Accuracy Score for C = 0.001: 0.787598944591029
VALIDATION Balanced Error Rate for C = 0.001: 0.3678571428571429

TRAIN Accuracy Score for C = 0.01: 0.8066006600660066
TRAIN Balanced Error Rate for C = 0.01: 0.19658703071672345
TEST Accuracy Score for C = 0.01: 0.783641160949868
TEST Balanced Error Rate for C = 0.01: 0.31323796548592187
VALIDATION Accuracy Score for C = 0.01: 0.7928759894459103
VA

For conviniences, I created a datafrmae to show the results, where the rows are train BER, test BER, and validation BER; and columns are value of parameter C.

In [136]:
dic = {}
dic['Train BER'] = train_ber
dic['Test BER'] = test_ber
dic['Validation BER'] = vali_ber

pd.DataFrame.from_dict(dic, orient='index', 
                       columns=[10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4])

,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0
Train BER,0.251263,0.175904,0.196587,0.159659,0.162730,0.161706,0.258635,0.163072,0.230683
Test BER,0.309435,0.308470,0.313238,0.316644,0.316644,0.297173,0.292405,0.317325,0.293086
Validation BER,0.398728,0.367857,0.382290,0.277202,0.319080,0.277886,0.384344,0.321135,0.387769


In [118]:
print('Best BER:',bestBER)
print('Best C:', bestC)
print('Best Model:', bestModel)

Best BER: 0.2772015655577299
Best C: 0.1
Best Model: LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


*Analysis*: Here I choose C = 0.1 as the best parameter for my best model. The reason is that when C = 0.1, the balanced error rate (BER) for validation set is the smallest.

## Question 5

Question: Compute the Fβ scores for β = 1, β = 0.1, and β = 10 for the above classifier, using C = 1 (on the test set) (1 mark).

#### Answer for Question 5:

In [129]:
for num in [1, 0.1, 10]:
    print('TEST Fβ scores for Beta = ' + str(num) +':', 
          metrics.fbeta_score(y_test, test_predictions_3, beta = num))

TEST Fβ scores for Beta = 1: 0.14213197969543148
TEST Fβ scores for Beta = 0.1: 0.08162087277764951
TEST Fβ scores for Beta = 10: 0.5495530509133307


## Question 7

Question: Following the stub code, compute the PCA basis on the training set. Report the first PCA component
(i.e., pca.components [0]) (1 mark).

#### Answer for Question 7:

In [139]:
pca = PCA()
pca.fit(X_train)
print('The first PCA component: ', pca.components_[0])

The first PCA component:  [-5.66194086e-19 -2.93791278e-08  3.09022418e-07  1.18939159e-06
  4.70743252e-06  2.44710258e-03 -1.01298099e-06  1.76546361e-06
  5.01973388e-06 -8.98610008e-07 -9.81724115e-08  2.63187931e-07
  1.29754258e-06  2.82880656e-07  1.76546361e-06  1.46676095e-03
  1.09125843e-06  5.44237993e-06  1.76546361e-06  3.27935544e-07
  6.55831910e-05  3.34752532e-08  2.35355899e-07  2.68856867e-07
  3.88441701e-07  7.39994780e-07  9.56972844e-07 -7.78591508e-05
  2.06501549e-06  4.21607925e-06 -1.34408394e-06  2.97861089e-07
 -6.71486060e-04  5.55451651e-06 -2.11686320e-06  2.07342347e-07
 -6.08831917e-07  6.09166474e-03 -5.53122726e-07  1.93394593e-07
  1.45849666e-06  4.03088086e-06  2.18182907e-07  1.04657649e-04
  3.90736605e-05 -5.51670404e-06  2.98766825e-06 -3.55291648e-04
  2.82683039e-07  2.61717909e-07  3.89105425e-06 -8.50048254e-07
 -1.81735711e-06  3.84611281e-06  2.15330571e-06  9.99977007e-01
  1.49609184e-07 -6.09288822e-08 -2.46028433e-07  7.08987975e-07

## Question 8

Question: Compute the validation and test BER of a model that uses just the first N components (i.e., dimensions) for N = 5, 10, . . . , 25, 30. Again use class weight=’balanced’ and C = 1.0 (2 marks).

#### Answer for Question 8:

In [140]:
test_ber_8 = []
vali_ber_8 = []

for num in np.arange(5, 31, 5):
    Xpca_train = numpy.matmul(X_train, pca.components_[:num].T) 
    Xpca_valid = numpy.matmul(X_vali, pca.components_[:num].T) 
    Xpca_test = numpy.matmul(X_test, pca.components_[:num].T)
    
    model = linear_model.LogisticRegression(C=1.0, class_weight='balanced')
    model.fit(Xpca_train, y_train)
    
    test_ber_8.append(1 - metrics.balanced_accuracy_score(y_test, model.predict(Xpca_test)))
    vali_ber_8.append(1 - metrics.balanced_accuracy_score(y_vali, model.predict(Xpca_valid)))
    
    print('TEST Balanced Error Rate for first ' + str(num) +' Component:', 
          1 - metrics.balanced_accuracy_score(y_test, model.predict(Xpca_test)))

    print('VALIDATION Balanced Error Rate for first ' + str(num) +' Component:', 
          1 - metrics.balanced_accuracy_score(y_vali, model.predict(Xpca_valid)))
    
    print('')

TEST Balanced Error Rate for first 5 Component: 0.40264532243415074
VALIDATION Balanced Error Rate for first 5 Component: 0.3234344422700587

TEST Balanced Error Rate for first 10 Component: 0.36960717529518616
VALIDATION Balanced Error Rate for first 10 Component: 0.4076810176125245

TEST Balanced Error Rate for first 15 Component: 0.32725930971843775
VALIDATION Balanced Error Rate for first 15 Component: 0.4056262230919765

TEST Balanced Error Rate for first 20 Component: 0.2704927338782924
VALIDATION Balanced Error Rate for first 20 Component: 0.36232876712328765

TEST Balanced Error Rate for first 25 Component: 0.301941416893733
VALIDATION Balanced Error Rate for first 25 Component: 0.37676125244618397

TEST Balanced Error Rate for first 30 Component: 0.2958106267029973
VALIDATION Balanced Error Rate for first 30 Component: 0.3877690802348337



For conviniences, I created a datafrmae to show the results, where the rows are test BER and validation BER, and columns are number of n first components.

In [141]:
dic_8 = {}
dic_8['Test BER'] = test_ber_8
dic_8['Validation BER'] = vali_ber_8

pd.DataFrame.from_dict(dic_8, orient='index', 
                       columns=[5,10,15,20,25,30])

,5,10,15,20,25,30
Test BER,0.402645,0.369607,0.327259,0.270493,0.301941,0.295811
Validation BER,0.323434,0.407681,0.405626,0.362329,0.376761,0.387769
